In [8]:
import pandas as pd
import scipy as sp
import numpy as np

import sklearn
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from numpy import arange
from sklearn import metrics


In [6]:
data = pd.read_csv('Resources/data_features.csv', sep='\t', index_col=0)
data = pd.DataFrame(data)
del data['Unnamed: 0.1']
del data['Unnamed: 0.1.1']
del data['\'\'']

del data['user']
data = data.fillna(0)
data.describe()

Yage = data['age']
Ysex = data['sex']

del data['age']
del data['sex']
X = data
print (X.columns)

scaler = StandardScaler()
X = scaler.fit_transform(X)
# X = pd.DataFrame(X)
# X.head()
len(X[0])

OSError: File b'Resources/data_features.csv' does not exist

In [219]:
# split data for AGE
X = pd.DataFrame(X)
Xage_train, Xage_test, Yage_train, Yage_test = train_test_split(X, Yage, test_size = 0.20, random_state = 23)

# split data for SEX
Xsex_train, Xsex_test, Ysex_train, Ysex_test = train_test_split(X, Ysex, test_size = 0.20, random_state = 23)

In [239]:
# build logistic regression model
# AGE
log_model_age = LogisticRegression()
log_model_age.fit(Xage_train, Yage_train)
print ('Logistic Regression Score for AGE: %.3f' % model_age.score(Xage_test, Yage_test))

log_model_sex = LogisticRegression()
log_model_sex.fit(Xsex_train, Ysex_train)
print ('Logistic Regression Score for SEX: %.3f' % model_sex.score(Xsex_test, Ysex_test))

joblib.dump(log_model_age, 'log_model_age.pkl', compress = 9)
joblib.dump(log_model_sex, 'log_model_sex.pkl', compress = 9)

Logistic Regression Score for AGE: 0.432
Logistic Regression Score for SEX: 0.580


['log_model_sex.pkl']

In [187]:
# build SVM model
# AGE
C_range = [1e-2, 1e-1, 1, 0.5, 1e1, 20, 30, 50, 1e2]
gamma_range = [1e-1, 1, 1e1]
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(Yage, n_iter=5, test_size=0.2, random_state=42)
grid_age_svm = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid_age_svm.fit(X, Yage)

print("The best parameters for AGE are %s with a score of %0.2f"
      % (grid_age_svm.best_params_, grid_age_svm.best_score_))

cv = StratifiedShuffleSplit(Yage, n_iter=5, test_size=0.2, random_state=42)
grid_sex_svm = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid_sex_svm.fit(X, Ysex)

print("The best parameters for AGE are %s with a score of %0.2f"
      % (grid_sex_svm.best_params_, grid_sex_svm.best_score_))

The best parameters for AGE are {'gamma': 0.1, 'C': 10.0} with a score of 0.46
The best parameters for AGE are {'gamma': 0.1, 'C': 10.0} with a score of 0.57


In [240]:
joblib.dump(grid_age_svm, 'grid_age_svm.pkl', compress = 9)
joblib.dump(grid_sex_svm, 'grid_sex_svm.pkl', compress = 9)

['grid_sex_svm.pkl']

In [34]:
# build GBM model
# AGE
learning_rate_range = [0.1, 0.2, 0.5]
n_estimators_range = [100, 150, 200]
max_depth_range = [3, 4]
param_grid = dict(learning_rate=learning_rate_range, n_estimators=n_estimators_range, max_depth=max_depth_range)
cv = StratifiedShuffleSplit(Yage, n_iter=5, test_size=0.2, random_state=42)
grid_age_gbm = GridSearchCV(GradientBoostingClassifier(), param_grid = param_grid, cv = cv)
grid_age_gbm.fit(X, Yage)
print("The best parameters for AGE are %s with a score of %0.2f"
      % (grid_age_gbm.best_params_, grid_age_gbm.best_score_))

NameError: name 'StratifiedShuffleSplit' is not defined

In [199]:
# build GBM model
# SEX
learning_rate_range = [0.1, 0.2, 0.5]
n_estimators_range = [100, 150, 200]
max_depth_range = [3, 4]
param_grid = dict(learning_rate=learning_rate_range, n_estimators=n_estimators_range, max_depth=max_depth_range)
cv = StratifiedShuffleSplit(Ysex, n_iter=5, test_size=0.2, random_state=42)
grid_sex_gbm = GridSearchCV(GradientBoostingClassifier(), param_grid = param_grid, cv = cv)
grid_sex_gbm.fit(X, Ysex)
print("The best parameters for SEX are %s with a score of %0.2f"
      % (grid_age_gbm.best_params_, grid_age_gbm.best_score_))

The best parameters for SEX are {'n_estimators': 100, 'learning_rate': 0.2, 'max_depth': 4} with a score of 0.42


In [33]:
joblib.dump(grid_age_gbm, 'grid_age_gbm.pkl', compress = 9)
joblib.dump(grid_sex_gbm, 'grid_sex_gbm.pkl', compress = 9)

NameError: name 'grid_age_gbm' is not defined

In [237]:
# Trying to predict 
features = [4, 2, 0, 0, 1, 35, 1, 0, 0, 8, 50.0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 
            0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
features = np.array(features).reshape(1, -1)
scaler = StandardScaler()
features = scaler.fit_transform(features)

print (grid_age_svm.predict(features))
print (model_sex.predict(features))

['25-34\r\r\r\r\r\n']
['MALE']
